### Heap-Hashmap Architecture

Note: since heap is not a stable sorting algorithm, we must order each element by both price and a timestamp. We also are not storing quantity in the heap, so the unique price and timestamp tuple must be some key in a hashmap storing the quantity available at that price. 

Currently I am not passing an order ID, nor an AuthorID for each order. In order to properly log the trades, we will need unique IDs for each order, each trade, and we will need to store all of those placing orders in another data structure, probably a hashmap, with clientID as the key and their actual alias as the value.

However, if we store only each available price level in the heap, we can maintain a priority queue associated with each price level, which will support linear amortized-time order book matching assuming there are an average of more than log(# of price levels) orders per price level.

### Matching Engine (Back-End)

The engine will be responsible for ingesting new trade/cancel requests, and sending out the current book when prompted.
- generates trade sequence numbers (order in which orders are received)
- generates trade timestamps
- generates execution_information (ID, timestamp, qty, price, seq)
- generates order cancel information (ID, timestamp, seq)
- maintains current orders in memory
- creates order/cancel log and writes to file in real time
- creates execution log and writes to file in real time


In [69]:
#matching engine... should be queried before placing an order into the book. So it's not added if not necessary
#matching engine will use the current order, and check the book for orders that it could be paired to. If 
#pair-able, fill pair-able orders until no more (either fully filled, or partial fill with no more available at
#a pair-able price leve).

#for efficiency, we want to query all orders at a matchable price level, in price-time priority, such that we can
#just loop through those until done with the fill

#matching engines store the order book in memory.. so the book class should contain the me.. or really the ME 
#class should contain the book..

import heapq
import pandas as pd
from datetime import datetime
import re

class Matching_Engine:
  def __init__(self):
    self.seq_num = 0
    self.bids = []
    self.offers = []
    self.order_info = {}
    self.seq_to_ID = {}
    self.order_log = {}
    self.execution_log = {}

  def new_order(self, direction, price, qty, order_ID):
    pattern = r"^[a-z]{2}\d{4}$"
    if direction not in 'BbOo' or price < 0 or qty < 0 or not re.fullmatch(pattern, order_ID):
      return
    timestamp = datetime.now().timestamp()
    seq = self.seq_num
    self.seq_num += 1
#order log
    self.order_log[order_ID] = [timestamp, direction.lower(), price, qty]
    if direction in 'bB':
      self.add_bid(price, qty, timestamp, order_ID, seq)
    else:
      self.add_offer(price, qty, timestamp, order_ID, seq)
    return
  
#NOTES: might be quicker to just check whether the best bid/offer price crosses witb
#the new order before redirecting to the match_bid/offer function. marginal reduction
#in complexity?
  def add_bid(self, price, qty, timestamp, order_ID, seq):
    if self.offers and self.match_bid(price, qty, timestamp, order_ID, seq):
      return
    heapq.heappush(self.bids, (-price, seq))
    self.seq_to_ID[seq] = order_ID
    self.order_info[order_ID] = [qty, seq, timestamp, -price]

  def add_offer(self, price, qty, timestamp, order_ID, seq):
    if self.bids and self.match_offer(price, qty, timestamp, order_ID, seq):
      return
    heapq.heappush(self.offers, (price, seq))
    self.seq_to_ID[seq] = order_ID
    self.order_info[order_ID] = [qty, seq, timestamp, price]

  def match_bid(self, price, qty, timestamp, order_ID, seq):
    min_offer = self.offers[0][0]
    min_offer_seq = self.offers[0][1]
    while min_offer_seq not in self.seq_to_ID:
      heapq.heappop(self.offers)
      if not self.offers:
        self.add_bid(self, price, qty, timestamp, order_ID, seq)
        return
      min_offer = self.offers[0][0]
      min_offer_seq = self.offers[0][1]
    min_offer_ID = self.seq_to_ID[min_offer_seq]
    if price >= min_offer:
      min_offer_qty = self.order_info[min_offer_ID][0]
      trade_size = min(min_offer_qty, qty)
      qty -= trade_size
      self.order_info[min_offer_ID][0] -= trade_size
      exec_timestamp = datetime.now().timestamp()
#execution log      
      exec_ID = 'EX' + str(self.seq_num).zfill(4)
      self.seq_num += 1
      buyer_ID = order_ID[:2]
      seller_ID = min_offer_ID[:2]      
# exec_ID : [timestamp, buyer_ID, bid_ID, seller_ID, offer_ID, price, qty]
      self.execution_log[exec_ID] = [exec_timestamp, buyer_ID, order_ID, seller_ID, seller_ID, min_offer, trade_size]

      print(f"Trade Executed: Bid ID = {order_ID}\tOffer ID = {min_offer_ID}size = {trade_size}\tprice = {min_offer}")

      if not self.order_info[min_offer_ID][0]:
        heapq.heappop(self.offers)
        del self.order_info[min_offer_ID]
        del self.seq_to_ID[min_offer_seq]
      if qty:
        self.add_bid(price, qty, timestamp, order_ID, seq)
      return True

    return False
    
  def match_offer(self, price, qty, timestamp, order_ID, seq):
    max_bid = -self.bids[0][0]
    max_bid_seq = self.bids[0][1]
    while max_bid_seq not in self.seq_to_ID:
      heapq.heappop(self.bids)
      if not self.bids:
        self.add_offer(self, price, qty, timestamp, order_ID, seq)
        return
      max_bid = -self.bids[0][0]
      max_bid_seq = self.bids[0][1]
    max_bid_ID = self.seq_to_ID[max_bid_seq]
    if price <= max_bid:
      max_bid_qty = self.order_info[max_bid_ID][0]
      trade_size = min(max_bid_qty, qty)
      qty -= trade_size
      self.order_info[max_bid_ID][0] -= trade_size
      exec_timestamp = datetime.now().timestamp()
#execution log      
      exec_ID = 'EX' + str(self.seq_num).zfill(4)
      self.seq_num += 1
      buyer_ID = max_bid_ID[:2]
      seller_ID = order_ID[:2]      
# exec_ID : [timestamp, buyer_ID, bid_ID, seller_ID, offer_ID, price, qty]
      self.execution_log[exec_ID] = [exec_timestamp, buyer_ID, max_bid_ID, seller_ID, order_ID, max_bid, trade_size]

#generate execution ID.. 
      print(f"Trade Executed: Bid ID = {max_bid_ID}\tOffer ID = {order_ID}\tsize = {trade_size}\tprice = {max_bid}")

      if not self.order_info[max_bid_ID][0]:
        heapq.heappop(self.bids)
        del self.order_info[max_bid_ID]
        del self.seq_to_ID[max_bid_seq]
      if qty:
        self.add_offer(price, qty, timestamp, order_ID, seq)
      return True

    return False

  def cancel_order(self, order_ID):
    if order_ID not in self.order_info:
      print(f"Error Log: Cancel Order Reject - No such order ID on book: [\'{order_ID}]\'")
      return
#add seq num, add to order/cancel log
    seq_num = self.order_info[order_ID][1]
    del self.order_info[order_ID]
    del self.seq_to_ID[seq_num]
  
  def get_book(self):
    bids_copy = self.bids.copy()
    offers_copy = self.offers.copy()
    rows = []

    while bids_copy or offers_copy:
      cur_row = []
      if bids_copy:
        b_price, b_seq = heapq.heappop(bids_copy)
        if b_seq not in self.seq_to_ID:
          continue
        b_ID = self.seq_to_ID[b_seq]
        b_qty = self.order_info[b_ID][0]
        cur_row.append(b_ID)
        cur_row.append(b_qty)
        cur_row.append(-b_price)
      else:
        cur_row.append('')
        cur_row.append('')
        cur_row.append('')
      
      if offers_copy:
        o_price, o_seq = heapq.heappop(offers_copy)
        if o_seq not in self.seq_to_ID:
          continue
        o_ID = self.seq_to_ID[o_seq]
        o_qty = self.order_info[o_ID][0]
        cur_row.append(o_price)
        cur_row.append(o_qty)
        cur_row.append(o_ID)
      else:
        cur_row.append('')
        cur_row.append('')
        cur_row.append('')
      rows.append(cur_row)

    book = pd.DataFrame(columns=['bid_ID', 'bid_qty', 'bid_price', 'offer_price', 'offer_qty', 'offer_ID'], data=rows)

    display(book)
    return

ME = Matching_Engine()

ME.new_order('o', 12, 100, 'aa0000')
ME.new_order('b', 11.50, 50, 'aa0001')
ME.new_order('b', 11.75, 100, 'aa0002')
ME.new_order('b', 11.5, 25, 'aa0003')
ME.cancel_order('aa0002')
ME.new_order('o', 12.05, 75, 'aa0004')
ME.new_order('o', 11.50, 70, 'aa0005')
ME.get_book()

Trade Executed: Bid ID = aa0001	Offer ID = aa0005	size = 50	price = 11.5
Trade Executed: Bid ID = aa0003	Offer ID = aa0005	size = 20	price = 11.5


,bid_ID,bid_qty,bid_price,offer_price,offer_qty,offer_ID
0,aa0003,5,11.5,12.00,100,aa0000
1,,,,12.05,75,aa0004


### Trading System (Front-End)

This system will be responsible for all non-matching-engine specific tasks, including:
- generating unique trade_ID's
- validating trade information

In [70]:
class Trading_System:
    def __init__(self, name, ID, ME):
        self.name = name
        self.ID = ID
        self.ME = ME
        self.sys_seq = 0

#NOTES: we are checking the direction already here, so maybe we can remove the new_order
#and go straight to the add_bid/offer
    def order(self, direction, price, qty):
        if not self.validate(direction, price, qty):
            return
        order_ID = self.new_ID()
        self.ME.new_order(direction, price, qty, order_ID)
    
    def new_ID(self):
        cur_sys_seq = str(self.sys_seq).zfill(4)
        self.sys_seq += 1
        order_ID = self.ID + cur_sys_seq
        return order_ID

#add checks for parameter types.. check for tick size on price? int on qty
    def validate(self, direction, price, qty):
        valid = True
        if direction not in 'bBoO':
            print(f"Error Log: Order Validation - Order Type Error: \tOrder [\'{o_type}\', {price}, {qty}]\n\
Invalid order type: \'{direction}\'.\t\t\t\tValid Orders are \'b\' or \'o\'.")
            valid = False
        if price <= 0:
            print(f"Error Log: Order Validation - Price Specification Error: \tOrder [\'{o_type}\', {price}, {qty}]\n\
Invalid Price: {price}.\t\t\t\tValid Prices are Greater than Zero (p > 0)")
            valid = False
        if qty <= 0:
            print(f"Error Log: Order Validation - Size Specification Error: \tOrder [\'{o_type}\', {price}, {qty}]\n\
Invalid Size: {price}.\t\t\t\tValid Sizes are Greater than Zero (qty > 0)")
            valid = False
        return valid

    def book(self):
        self.ME.get_book()

### Sample Usage

initialize matching engine
each instance of trading system can represent a different firm

Current throughput ~= 3000 orders/s with no matching..
matching..

In [85]:
ME = Matching_Engine()

op = Trading_System('Optiver', 'op', ME)
js = Trading_System('Jane Street', 'js', ME)

op.order('o', 12, 100)
js.order('b', 11.75, 10)
js.order('b', 11.75, 10)
op.order('b', 11.75, 10)
op.order('b', 11.75, 10)
op.order('o', 12, 100)
js.order('b', 11.75, 10)
js.order('b', 11.75, 10)
op.order('b', 11.75, 10)
op.order('b', 11.75, 10)
js.order('o', 11.75, 100)

ME.get_book()

print(ME.execution_log)

prev = None
dt = 0
print(ME.execution_log.values())
for time, buyer_ID, bid_ID, seller_ID, offer_ID, price, qty in list(ME.execution_log.values()):
    if not prev: 
        prev = time
        continue
    print(time, prev, time-prev)
    dt += (time-prev)
    prev = time

print(1/dt)

prev = None
dt = 0
print(ME.order_log.values())
for time, type, qty, price in list(ME.order_log.values()):
    if not prev: 
        prev = time
        continue
    print(time, prev, time-prev)
    dt += (time-prev)
    prev = time

print(1/dt)

Trade Executed: Bid ID = js0000	Offer ID = js0004	size = 10	price = 11.75
Trade Executed: Bid ID = js0001	Offer ID = js0004	size = 10	price = 11.75
Trade Executed: Bid ID = op0001	Offer ID = js0004	size = 10	price = 11.75
Trade Executed: Bid ID = op0002	Offer ID = js0004	size = 10	price = 11.75
Trade Executed: Bid ID = js0002	Offer ID = js0004	size = 10	price = 11.75
Trade Executed: Bid ID = js0003	Offer ID = js0004	size = 10	price = 11.75
Trade Executed: Bid ID = op0004	Offer ID = js0004	size = 10	price = 11.75
Trade Executed: Bid ID = op0005	Offer ID = js0004	size = 10	price = 11.75


,bid_ID,bid_qty,bid_price,offer_price,offer_qty,offer_ID
0,,,,11.75,20,js0004
1,,,,12.00,100,op0000
2,,,,12.00,100,op0003


{'EX0011': [1734359155.021772, 'js', 'js0000', 'js', 'js0004', 11.75, 10], 'EX0012': [1734359155.021998, 'js', 'js0001', 'js', 'js0004', 11.75, 10], 'EX0013': [1734359155.02202, 'op', 'op0001', 'js', 'js0004', 11.75, 10], 'EX0014': [1734359155.022034, 'op', 'op0002', 'js', 'js0004', 11.75, 10], 'EX0015': [1734359155.022044, 'js', 'js0002', 'js', 'js0004', 11.75, 10], 'EX0016': [1734359155.022055, 'js', 'js0003', 'js', 'js0004', 11.75, 10], 'EX0017': [1734359155.022065, 'op', 'op0004', 'js', 'js0004', 11.75, 10], 'EX0018': [1734359155.022075, 'op', 'op0005', 'js', 'js0004', 11.75, 10]}
dict_values([[1734359155.021772, 'js', 'js0000', 'js', 'js0004', 11.75, 10], [1734359155.021998, 'js', 'js0001', 'js', 'js0004', 11.75, 10], [1734359155.02202, 'op', 'op0001', 'js', 'js0004', 11.75, 10], [1734359155.022034, 'op', 'op0002', 'js', 'js0004', 11.75, 10], [1734359155.022044, 'js', 'js0002', 'js', 'js0004', 11.75, 10], [1734359155.022055, 'js', 'js0003', 'js', 'js0004', 11.75, 10], [1734359155.